In [1]:
import numpy as np
import scipy.sparse as sp
import pandas as pd

In [2]:
file_args = "proj1_parameter-file.txt"
file_data = "proj1_input-data.txt"
#file_args = "proj1_ex13_args.txt"
#file_data = "proj1_ex13_data.txt"

In [3]:
ms = dict()
sdc = 1
x_exclude = []
x_must = []
for i in open(file_args, "r"):
    i = i.rstrip("\n")
    if i.startswith("MIS"):
        j = i.split(" = ")
        ms.update({j[0][4:-1]: float(j[1])})
    elif i.startswith("SDC"):
        sdc = float(i.split("=")[1])
    elif i.startswith("cannot_be_together"):
        x_exclude = [j.split(", ") for j in i.split(": ")[1][1:-1].split("}, {")]
    elif i.startswith("must"):
        x_must = [j for j in i.split(": ")[1].split(" or ")]
ms, sdc, x_exclude, x_must

({'10': 0.43,
  '100': 0.1,
  '120': 0.2,
  '140': 0.15,
  '20': 0.3,
  '30': 0.3,
  '40': 0.4,
  '50': 0.4,
  '60': 0.3,
  '70': 0.2,
  '80': 0.2,
  '90': 0.2},
 0.1,
 [['20', '40'], ['70', '80']],
 ['20', '40', '50'])

In [4]:
ms = pd.Series(ms, name = "MIS").sort_values().reset_index()
ms

,index,MIS
0,100,0.10
1,140,0.15
2,120,0.20
3,70,0.20
4,80,0.20
5,90,0.20
6,20,0.30
7,30,0.30
8,60,0.30
9,40,0.40


In [5]:
id_dict = ms["index"].to_dict()
id_dict_inv = {val: key for key, val in id_dict.items()}
x_must = [id_dict_inv[i] for i in x_must]
x_exclude = [tuple(np.sort([id_dict_inv[j] for j in i])) for i in x_exclude]
id_dict, x_must, x_exclude

({0: '100',
  1: '140',
  2: '120',
  3: '70',
  4: '80',
  5: '90',
  6: '20',
  7: '30',
  8: '60',
  9: '40',
  10: '50',
  11: '10'},
 [6, 9, 10],
 [(6, 9), (3, 4)])

In [6]:
ms_dict = ms["MIS"].to_dict()
ms_dict

{0: 0.10000000000000001,
 1: 0.14999999999999999,
 2: 0.20000000000000001,
 3: 0.20000000000000001,
 4: 0.20000000000000001,
 5: 0.20000000000000001,
 6: 0.29999999999999999,
 7: 0.29999999999999999,
 8: 0.29999999999999999,
 9: 0.40000000000000002,
 10: 0.40000000000000002,
 11: 0.42999999999999999}

In [7]:
s = pd.read_csv(file_data, header = None, sep = "\t",squeeze = True)
s

0                   {20, 30, 80, 70, 50, 90}
1                           {20, 10, 80, 70}
2                               {10, 20, 80}
3                               {20, 30, 80}
4                                   {20, 80}
5    {20, 30, 80, 70, 50, 90, 100, 120, 140}
Name: 0, dtype: object

In [8]:
da = s.str[1:-1].str.get_dummies(sep = ", ").reindex(columns = ms["index"], fill_value = 0).values
da

array([[0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0],
       [0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0]])

In [9]:
Isup = da.mean(axis = 0)
Isup

array([ 0.16666667,  0.16666667,  0.16666667,  0.5       ,  1.        ,
        0.33333333,  1.        ,  0.5       ,  0.        ,  0.        ,
        0.33333333,  0.33333333])

In [10]:
sup_dict = dict(zip([(i,) for i in range(len(ms))], Isup))
sup_dict

{(0,): 0.16666666666666666,
 (1,): 0.16666666666666666,
 (2,): 0.16666666666666666,
 (3,): 0.5,
 (4,): 1.0,
 (5,): 0.33333333333333331,
 (6,): 1.0,
 (7,): 0.5,
 (8,): 0.0,
 (9,): 0.0,
 (10,): 0.33333333333333331,
 (11,): 0.33333333333333331}

In [11]:
Li = (Isup > ms["MIS"]).argmax()
L = [i for i in range(Li, ms.shape[0]) if Isup[i] > ms["MIS"][Li]]
Li, L

(0, [0, 1, 2, 3, 4, 5, 6, 7, 10, 11])

In [12]:
F = [[(i,) for i in np.where(Isup > ms["MIS"])[0]]]
F

[[(0,), (1,), (3,), (4,), (5,), (6,), (7,)]]

In [13]:
def mis(xL):
    op = [ms_dict[i[0]] for i in xL]
    return(op)
def sup(xL):
    op = np.mean([da[:, i].all(axis = 1) for i in xL], axis = 1)
    return(op) 
def frequent(C):
    C_sup = sup(C)
    sup_dict.update(dict(zip(C, C_sup)))
    op = [C[i] for i in np.where(C_sup >= mis(C))[0]]
    C_dropfirst = set(tuple(i[1:]) for i in C)
    sup_dict.update(dict(zip(C_dropfirst, sup(C_dropfirst))))
    return(op)
def pair_sup_mis(x):
    x_t = x[:, np.newaxis]
    x_sup = sup(x_t)
    x_mis = mis(x_t)
    x_sup_t = x_sup[:, np.newaxis]
    C = sp.coo_matrix(np.triu((x_sup_t >= x_mis).T & (np.abs(x_sup_t - x_sup) < sdc), 1)).nonzero()
    op = list(zip(x[C[0]], x[C[1]]))
    return(op)
def append_set(C, base):
    if len(C):
        op = [tuple(i) for i in np.hstack([np.tile(base, (len(C), 1)), C])]
    else:
        op = []
    return(op)
def prune_candidate(C):
    op = [C[i] for i in np.where(np.all([[any(set(k).issubset(j) for k in F[-1]) for j in np.delete(C, i, axis = 1)] for i in range(1, len(C[0]))], axis = 0))[0]]
    return(op)
def id_name(xL):
    op = [tuple(id_dict[j] for j in i) for i in xL]
    return(op)

In [14]:
C = [i for i in pair_sup_mis(np.array(L)) if i[0] in np.array(F[0]).T[0]]
C

[(0, 1), (0, 2), (1, 2), (3, 7), (4, 6), (5, 10), (5, 11)]

In [15]:
F.append(frequent(C))
F

[[(0,), (1,), (3,), (4,), (5,), (6,), (7,)],
 [(0, 1), (0, 2), (1, 2), (3, 7), (4, 6), (5, 10)]]

In [16]:
Ls = pd.DataFrame(F[-1])
Ls

,0,1
0,0,1
1,0,2
2,1,2
3,3,7
4,4,6
5,5,10


In [17]:
C = sum([append_set(pair_sup_mis(group.values), name) for name, group in Ls.groupby(list(range(len(F)-1)))[len(F)-1]], [])
C = prune_candidate(C)
C

[(0, 1, 2)]

In [18]:
F.append(frequent(C))
F

[[(0,), (1,), (3,), (4,), (5,), (6,), (7,)],
 [(0, 1), (0, 2), (1, 2), (3, 7), (4, 6), (5, 10)],
 [(0, 1, 2)]]

In [19]:
F_prune = [[j for j in i if any(k in j for k in x_must) & ~any(set(k).issubset(j) for k in x_exclude)] for i in F]
F_prune

[[(6,)], [(4, 6), (5, 10)], []]

In [20]:
[id_name(i) for i in F_prune]

[[('20',)], [('80', '20'), ('90', '50')], []]

In [21]:
sup_dict

{(0,): 0.16666666666666666,
 (0, 1): 0.16666666666666666,
 (0, 1, 2): 0.16666666666666666,
 (0, 2): 0.16666666666666666,
 (1,): 0.16666666666666666,
 (1, 2): 0.16666666666666666,
 (2,): 0.16666666666666666,
 (3,): 0.5,
 (3, 7): 0.33333333333333331,
 (4,): 1.0,
 (4, 6): 1.0,
 (5,): 0.33333333333333331,
 (5, 10): 0.33333333333333331,
 (5, 11): 0.0,
 (6,): 1.0,
 (7,): 0.5,
 (8,): 0.0,
 (9,): 0.0,
 (10,): 0.33333333333333331,
 (11,): 0.33333333333333331}